In [1]:
import os
from tifffile import imread
import zarr
from ome_zarr.io import parse_url
from ome_zarr.writer import write_image
from ome_zarr.reader import Reader
import napari
from dask import delayed
import dask.array as da

In [2]:
im_path = r'D:\data_analysis\2023_Stallaert\TP19M626-22_Final\TP19M626-22_1.0.4_R000_Cy7_CDC6-AF750_FINAL_AFR_F.tif'
zarr_path = r'D:\data_analysis\2023_Stallaert\TP19M626-22_Final\test.zarr'

## Save zarr 

In [35]:
im = imread(im_path)
im.shape

(40696, 57410)

In [39]:
size_xy = 128

# write the image data
store = parse_url(zarr_path, mode="w").store
root = zarr.group(store=store)

# it will fail if the store already contains arrays
write_image(image=im, group=root, axes="yx", storage_options=dict(chunks=(size_xy, size_xy)))

## Zarr to dask

In [ ]:
reader = Reader(parse_url(zarr_path))
image_node = list(reader())[0]

dask_data = image_node.data

In [38]:
# visualize dask data
dask_data[0]

dask.array<from-zarr, shape=(40696, 57410), dtype=uint16, chunksize=(128, 128), chunktype=numpy.ndarray>

In [37]:
viewer = napari.Viewer()
viewer.add_image(dask_data)

<Image layer 'dask_data' at 0x1640651c880>

## Stack of tiffs to dask

In [3]:
im_dir = r'D:\data_analysis\2023_Stallaert\TP19M626-22_Final'
filenames_org = ['TP19M626-22_1.0.4_R000_DAPI__FINAL_F.tif','TP19M626-22_1.0.4_R000_Cy7_CDC6-AF750_FINAL_AFR_F.tif']
filenames = [os.path.join(im_dir,i) for i in filenames_org]

# read the first file to get the shape and dtype
# ASSUMES THAT ALL FILES SHARE THE SAME SHAPE/TYPE
sample = imread(filenames[0])

lazy_imread = delayed(imread)  # lazy reader
lazy_arrays = [lazy_imread(fn) for fn in filenames]
dask_arrays = [
    da.from_delayed(delayed_reader, shape=sample.shape, dtype=sample.dtype)
    for delayed_reader in lazy_arrays
]

# Stack into one large dask.array
stack = da.stack(dask_arrays, axis=0)
stack.shape  # (nfiles, nz, ny, nx)

# in jupyter notebook the repr of a dask stack provides a useful visual:
stack

dask.array<stack, shape=(2, 40696, 57410), dtype=uint16, chunksize=(1, 40696, 57410), chunktype=numpy.ndarray>